In [1]:
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
import random 
from tqdm import tqdm
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.figure_factory as ff

In [12]:
# asset_list = ['MSFT', 'WMT']
asset_list = ['MSFT', 'WMT', 'JPM', 'TGT', 'AAPL', 'GOOGL', 'ABT', 'KO', 'AMZN', 'NVDA']
# asset_list = ['MSFT', 'WMT', 'JPM']
# asset_list = ['MSFT', 'AAPL', 'GOOGL', 'AMZN', 'NVDA']

# set the risk-free rate
risk_free_rate = 0.04

#-- How many portfolios to generate, a standard PC might start to struggle at 100,000
n_portfolios = 1000

sd = "2015-01-01"
ed = "2022-12-31"
int = "1d"

def create_dataframe(ticker):
    df = yf.download(ticker, start=sd, end=ed, interval=int)
    df = df.reset_index(level=0)
    df = df[["Date", "Adj Close"]]
    df = df.rename({'Adj Close': "price"}, axis=1)  # new method
    df["ret"] = (df.price - (df.price.shift(1)))/df.price.shift(1)
    df = df.drop('price', axis=1)
    df = df.rename({'ret': ticker}, axis=1)  # new method
    return df


daily_returns = create_dataframe(asset_list[0])
for i in asset_list[1:]:
    df2 = create_dataframe(i)
    daily_returns = daily_returns.merge(df2, how='inner', on='Date')
print (daily_returns.head())
daily_returns = daily_returns.drop('Date', axis=1)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
        Date      MSFT       WMT
0 2015-01-02       NaN       NaN
1 2015-01-05 -0.009196 -0.002910
2 2015-01-06 -0.014677  0.007705
3 2015-01-07  0.012705  0.026533
4 2015-01-08  0.029418  0.021107


In [15]:
x = daily_returns.head()
print (x)
print (x.cov())
print (x.var())

       MSFT       WMT
0       NaN       NaN
1 -0.009196 -0.002910
2 -0.014677  0.007705
3  0.012705  0.026533
4  0.029418  0.021107
          MSFT       WMT
MSFT  0.000415  0.000211
WMT   0.000211  0.000177
MSFT    0.000415
WMT     0.000177
dtype: float64


In [3]:
#-- Get annualised mean returns
mus = (1+daily_returns.mean())**252 - 1
# mus = (daily_returns.mean())*252

#-- Get covariances
#- Multiply by 252 to annualise it (square root time for volatility but no square root for variance)
#- Note: 252 trading days in a year
#- https://quant.stackexchange.com/questions/4753/annualized-covariance
cov = daily_returns.cov()*252
var = daily_returns.var()*252


In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=var, y=mus, 
                      mode='markers',
                      text = var.axes[0].tolist()))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Assets',
                  width=850,
                  height=500)
# fig.update_xaxes(range=[0.18, 0.32])
# fig.update_yaxes(range=[0.02,0.27])

In [5]:
#- How many assests to include in each portfolio
n_assets = len(asset_list)


#-- Initialize empty list to store mean-variance pairs for plotting
mean_variance_pairs = []

np.random.seed(75)
#-- Loop through and generate lots of random portfolios
for i in range(n_portfolios):
    #- Choose assets randomly without replacement
    assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
    #- Choose weights randomly
    weights = np.random.rand(n_assets)
    #- Ensure weights sum to 1
    weights = weights/sum(weights)

    #-- Loop over asset pairs and compute portfolio return and variance
    #- https://quant.stackexchange.com/questions/43442/portfolio-variance-explanation-for-equation-investments-by-zvi-bodie
    portfolio_E_Variance = 0
    portfolio_E_Return = 0
    for i in range(len(assets)):
        portfolio_E_Return += weights[i] * mus.loc[assets[i]]
        for j in range(len(assets)):
            #-- Add variance/covariance for each asset pair
            #- Note that when i==j this adds the variance
            portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]
            
    #-- Add the mean/variance pairs to a list for plotting
    mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
    

In [6]:
#-- Plot the risk vs. return of randomly generated portfolios
#-- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers'))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
# fig.update_xaxes(range=[0, 0.4])
# fig.update_yaxes(range=[0,0.4])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))

In [7]:
#-- Create random portfolio weights and indexes

mean_variance_pairs = []
weights_list=[]
tickers_list=[]

for i in tqdm(range(10000)):
    next_i = False
    while True:
        #- Choose assets randomly without replacement
        assets = np.random.choice(list(daily_returns.columns), n_assets, replace=False)
        #- Choose weights randomly ensuring they sum to one
        weights = np.random.rand(n_assets)
        weights = weights/sum(weights)

        #-- Loop over asset pairs and compute portfolio return and variance
        portfolio_E_Variance = 0
        portfolio_E_Return = 0
        for i in range(len(assets)):
            portfolio_E_Return += weights[i] * mus.loc[assets[i]]
            for j in range(len(assets)):
                portfolio_E_Variance += weights[i] * weights[j] * cov.loc[assets[i], assets[j]]

        #-- Skip over dominated portfolios
        for R,V in mean_variance_pairs:
            if (R > portfolio_E_Return) & (V < portfolio_E_Variance):
                next_i = True
                break
        if next_i:
            break

        #-- Add the mean/variance pairs to a list for plotting
        mean_variance_pairs.append([portfolio_E_Return, portfolio_E_Variance])
        weights_list.append(weights)
        tickers_list.append(assets)
        break

100%|██████████| 10000/10000 [00:25<00:00, 391.58it/s]


In [66]:
#-- Plot the risk vs. return of randomly generated portfolios
#-- Convert the list from before into an array for easy plotting
mean_variance_pairs = np.array(mean_variance_pairs)

fig = go.Figure()
fig.add_trace(go.Scatter(x=mean_variance_pairs[:,1]**0.5, y=mean_variance_pairs[:,0], 
                      marker=dict(color=(mean_variance_pairs[:,0]-risk_free_rate)/(mean_variance_pairs[:,1]**0.5), 
                                  showscale=True, 
                                  size=7,
                                  line=dict(width=1),
                                  colorscale="RdBu",
                                  colorbar=dict(title="Sharpe<br>Ratio")
                                 ), 
                      mode='markers',
                      text=[str(np.array(tickers_list[i])) + "<br>" + str(np.array(weights_list[i]).round(2)) for i in range(len(tickers_list))]))
fig.update_layout(template='plotly_white',
                  xaxis=dict(title='Annualised Risk (Volatility)'),
                  yaxis=dict(title='Annualised Return'),
                  title='Sample of Random Portfolios',
                  width=850,
                  height=500)
# fig.update_xaxes(range=[0, 0.4])
# fig.update_yaxes(range=[0,0.4])
fig.update_layout(coloraxis_colorbar=dict(title="Sharpe Ratio"))